In [6]:
import pandas as pd
import os, time
import StkDataKeeper as sk

# determine 涨停
def isZhangting(s_df, index=-1):
    if (s_df.iloc[index].close/s_df.iloc[index-1].close) >= 1.099:
        return True
    else:
        return False

#determine 上涨
def isShangzhang(s_df, index=-1):
    if len(s_df.index) >= (index)+1:
        return ((s_df.iloc[index].close/s_df.iloc[index-1].close-1) > 0.02)
    else:
        return False

#determine 条件一
def isQualifiedCondition1(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+2):
        if (s_df.iloc[index].close * s_df.iloc[index].volume > 200000):
            return  (isZhangting(s_df, index-1) and isShangzhang(s_df,index))
    return False

#determine 条件二
def isQualifiedCondition2(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3):
        if (s_df.iloc[index].close / s_df.iloc[index-2].close < 0.92) \
                and (s_df.iloc[index].close<s_df.iloc[index-1].close) \
                and (s_df.iloc[index-1].close<s_df.iloc[index-2].close)\
                and isZhangting(s_df, index-2):
            return True
    return False

#determine 条件三
def isQualifiedCondition3(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3): 
        if isQualifiedCondition1(s_df, index-1): 
            if (s_df.iloc[index].close / s_df.iloc[index-1].close < 0.95):
                return True
        return False
        
def completeCode(code_str):
    if len(code_str)<6:
        return "0"*(6-len(code_str))+code_str
    else:
        return code_str

print (time.strftime("%M:%S"))
    
########################################
datafolder_dir = os.getcwd()+"/current"
########################################

file_list = os.listdir(datafolder_dir)

trading = -2
post_analysis = -1

analysis_type = trading

file_index = 0
select_list = []
file_dict = {}
for file in file_list:
    #print ("Working on file %s: %s"%(file_index, file))
    file_index = file_index + 1
    if ("day" in file):
        stock_df = pd.read_csv(datafolder_dir+"/"+file)
        if isQualifiedCondition1(stock_df, analysis_type):
            select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition2(stock_df, analysis_type):
            select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition3(stock_df, analysis_type):
            select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

select_list.sort()

# ------------------------------
for file in select_list:
    print (file)

for index in range(len(select_list)):
    select_list[index] = select_list[index][-6:]

print (select_list)

stock_df_list = []
for item in select_list:
    stock_df_list.append(pd.read_csv(file_dict[item]))

# ------------------------------



painter = sk.StkDiagram()
keeper = sk.StkDataKeeper()

keeper.get_stock_basics_tu("stock_basics.csv")

keeper.load_stock_list("stock_basics.csv")
klines = painter.paint_klines(select_list, "stock_basics.csv")

klines.width = 1600
klines.render("lyd_result/render.html")
klines.width = 990
klines


26:26
condition 1:002016
condition 1:002052
condition 1:002134
condition 1:002181
condition 1:002195
condition 1:002806
condition 1:002864
condition 1:002877
condition 1:002903
condition 1:002913
condition 1:002931
condition 1:300056
condition 1:300293
condition 1:300333
condition 1:300348
condition 1:300366
condition 1:300451
condition 1:300509
condition 1:300532
condition 1:300555
condition 1:300563
condition 1:300565
condition 1:300584
condition 1:300588
condition 1:300597
condition 1:300659
condition 1:300663
condition 1:300670
condition 1:300690
condition 1:300705
condition 1:300715
condition 1:300718
condition 1:600486
condition 1:601518
condition 1:603011
condition 1:603027
condition 1:603127
condition 1:603214
condition 1:603301
condition 1:603320
condition 1:603387
condition 1:603638
condition 1:603716
condition 2:000816
['002016', '002052', '002134', '002181', '002195', '002806', '002864', '002877', '002903', '002913', '002931', '300056', '300293', '300333', '300348', '300366

In [3]:
import tushare as ts
for index in range(len(select_list)):
    df = ts.get_realtime_quotes(select_list[index])
    # print (float(stock_df_list[index].tail(1).close), float(df.iloc[0]["price"]))
    ratio = (float(df.iloc[0]["price"]) / float(stock_df_list[index].tail(1).close))
    if ratio < 0.97 and ratio > 0.85:
        print (select_list[index], "diff is %s"%((1-ratio)*100))

AttributeError: 'NoneType' object has no attribute 'iloc'

In [4]:
klines.render("lyd_result/render.html")

In [4]:
keeper.get_stock_basics_tu("stock_basics.csv")

NameError: name 'keeper' is not defined

In [5]:
keeper = sk.StkDataKeeper()

keeper.get_stock_basics_tu("stock_basics.csv")

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
600866,星湖科技,食品,广东,97.41,6.45,6.45,142173.52,46828.85,73611.05,76112.06,...,1.43,3.35,19940818,-65730.11,-1.02,104.82,130.51,18.81,3.43,42532.0
000677,恒天海龙,化纤,山东,1610.23,8.64,8.64,84475.02,28404.76,38950.51,69768.81,...,0.32,11.42,19961226,-150178.95,-1.74,14.03,-57.28,12.09,0.35,53200.0
600710,苏美达,工程机械,江苏,26.21,6.40,13.07,4360007.50,3367155.50,733925.44,124962.77,...,3.11,2.00,19960701,129667.38,0.99,4.88,7.59,5.07,0.45,36090.0
600601,方正科技,电脑设备,上海,0.00,21.95,21.95,1106596.50,523075.31,337841.63,34737.42,...,1.69,2.01,19901219,110748.82,0.50,-20.36,-71.02,17.81,-5.86,245466.0
002280,联络互动,软件服务,浙江,342.96,18.50,21.77,1398823.00,750404.88,106541.57,314105.16,...,2.80,2.50,20090821,67218.06,0.31,796.54,-87.31,17.46,0.43,157044.0
000810,创维数字,家用电器,四川,35.64,9.60,10.71,780476.81,652062.56,68203.74,41705.40,...,2.61,3.45,19980602,141868.81,1.32,11.24,119.01,16.14,3.68,52820.0
600571,信雅达,软件服务,浙江,0.00,4.24,4.40,217717.91,110003.57,6078.98,19417.84,...,3.41,3.21,20021101,23198.92,0.53,-4.56,-270.35,50.36,-15.80,58652.0
600857,宁波中百,百货,浙江,58.11,2.24,2.24,81149.06,45711.68,11401.28,6507.80,...,0.79,14.71,19940425,-17658.07,-0.79,4.46,9.58,13.40,4.14,13916.0
002771,真视通,软件服务,北京,47.35,0.72,1.62,115356.42,99364.88,13428.30,19997.50,...,3.99,5.09,20150629,26123.17,1.62,3.03,0.36,25.44,8.46,16800.0


In [9]:
analysis_type = trading

file_index = 0
select_list = []
file_dict = {}
for file in file_list:
    #print ("Working on file %s: %s"%(file_index, file))
    file_index = file_index + 1
    if ("day" in file):
        stock_df = pd.read_csv(datafolder_dir+"/"+file)
        if isQualifiedCondition1(stock_df, analysis_type):
            select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition2(stock_df, analysis_type):
            select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition3(stock_df, analysis_type):
            select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

select_list.sort()

# ------------------------------
for file in select_list:
    print (file)

for index in range(len(select_list)):
    select_list[index] = select_list[index][-6:]

print (select_list)

stock_df_list = []
for item in select_list:
    stock_df_list.append(pd.read_csv(file_dict[item]))

# ------------------------------

painter1 = sk.StkDiagram()
keeper1 = sk.StkDataKeeper()
keeper1.load_stock_list("stock_basics.csv")
klines1 = painter1.paint_klines(select_list, "stock_basics.csv")

klines1.width = 1600
klines1.render("lyd_result/render1.html")
klines1.width = 990
klines1

condition 1:000928
condition 1:002898
condition 1:002907
condition 1:002910
condition 1:002916
condition 1:300022
condition 1:300349
condition 1:300697
condition 1:600520
condition 1:600903
condition 1:601038
condition 1:603363
condition 1:603367
condition 1:603386
condition 1:603676
condition 1:603963
['000928', '002898', '002907', '002910', '002916', '300022', '300349', '300697', '600520', '600903', '601038', '603363', '603367', '603386', '603676', '603963']


In [7]:
select_list = ['000911', '000916', '002809', '300428', '300500', '300725', '600249', '601069', '601933', '603986', '002302', '002912', '300666', '300727', '000504', '000759', '002419']

In [3]:

import StkDataKeeper as sk

keeper = sk.StkDataKeeper()
keeper.get_stock_basics_tu("stock_basics.csv")

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
002610,爱康科技,电气设备,江苏,59.40,42.79,44.91,1815202.38,601049.19,884513.13,86114.43,...,1.28,2.13,20110815,41977.26,0.09,20.83,90.37,18.00,4.34,232603.0
002587,奥拓电子,元器件,深圳,40.83,3.72,6.11,155489.67,111025.17,16139.77,18101.57,...,1.78,4.36,20110610,27295.89,0.45,134.20,46.58,38.74,13.29,33381.0
600353,旭光股份,元器件,四川,91.21,5.44,5.44,166766.48,129399.65,25157.61,11375.75,...,1.99,3.47,20021120,32345.08,0.59,12.55,-9.14,17.27,3.98,53097.0
600246,万通地产,区域地产,北京,68.71,12.17,20.54,1415504.75,1022710.38,5599.93,354743.72,...,3.27,1.54,20000922,93289.18,0.45,-18.34,-46.77,50.75,8.67,62984.0
002447,壹桥股份,互联网,辽宁,122.63,9.64,14.27,324293.28,72325.06,142711.44,10943.57,...,1.89,4.17,20100713,103880.61,0.73,-20.87,-47.62,75.60,29.69,54137.0
002591,恒大高新,化工原料,江西,0.00,1.76,3.07,144139.25,54440.16,26027.43,82812.02,...,4.16,2.66,20110621,8534.92,0.28,51.44,46.80,50.24,-4.21,16202.0
002369,卓翼科技,通信设备,深圳,418.58,4.20,5.81,339044.44,172945.55,93482.56,128657.27,...,3.66,3.18,20100316,17681.47,0.30,-6.23,26.31,9.16,0.65,30000.0
600571,信雅达,软件服务,浙江,151.54,4.24,4.40,169000.06,109965.07,5983.27,19457.15,...,2.64,4.86,20021101,46877.45,1.07,2.25,-67.48,50.90,3.37,59605.0
603386,广东骏亚,元器件,广东,55.58,0.51,2.02,121684.05,75937.71,34713.30,24305.42,...,2.97,6.06,20170912,14474.07,0.72,0.00,0.00,17.41,6.95,39743.0
